In [1]:
import pandas as pd

In [2]:
from text_dedup.near_dedup import SimHashEmbedder
from text_dedup.postprocess import simhash_clustering
from text_dedup.postprocess import get_group_indices

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 5.07M/5.07M [00:00<00:00, 72.2MB/s]
Downloading: 100%|██████████| 9.10M/9.10M [00:00<00:00, 69.1MB/s]
Downloading: 100%|██████████| 615/615 [00:00<00:00, 449kB/s]


In [3]:
RAW_DATA_PATH = '/work/data/train_prepcroc.csv'
PREPROC_DATA_PATH = '/work/data/train_prepcroc.csv'

In [4]:
df = pd.read_csv(RAW_DATA_PATH)

In [7]:
df.columns

Index(['pair_id', 'name_1', 'name_2', 'is_duplicate', 'fuzz_ratio',
       'fuzz_part_ratio', 'fuzz_set_ratio', 'fuzz_sort_ratio'],
      dtype='object')

In [21]:
simembedder = SimHashEmbedder()

def create_simhashembedding(name):
    return simembedder.embed_function()(name)


In [22]:
text = 'Iko Industries Ltd.'
print(simembedder.embed_function()(text))


8280139831965439638


In [23]:
df['name_1_simhash'] = df.apply(lambda x: create_simhashembedding(x.name_1), axis=1)

In [25]:
df['name_2_simhash'] = df.apply(lambda x: create_simhashembedding(x.name_2), axis=1)

In [39]:
embedder = MinHashEmbedder() #num_perm=16

def create_minhashembedding(name):
    return embedder.embed_function()(name)

In [41]:
df['name_1_minhash'] = df.apply(lambda x: create_minhashembedding(x.name_1), axis=1)
df['name_2_minhash'] = df.apply(lambda x: create_minhashembedding(x.name_2), axis=1)

In [44]:
df.head(2)

,pair_id,name_1,name_2,is_duplicate,fuzz_ratio,fuzz_part_ratio,fuzz_set_ratio,fuzz_sort_ratio,name_1_simhash,name_2_simhash,name_1_minhash,name_2_minhash
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0,58,68,59,55,8280139831965439638,1189005557588197376,"[2837697543, 4286705991, 4033516562, 157688461...","[2269330533, 1950295083, 2747852593, 821000859..."
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0,62,65,78,59,3467845109336244224,10454710756391821586,"[383969271, 2548742056, 266325389, 3073401718,...","[41399270, 2875731467, 1607583424, 1048751950,..."


In [47]:
def get_groups(name_1_emb, name_2_emb):

    clusters = lsh_clustering((name_1_emb, name_2_emb), seed=1)
    groups = get_group_indices(clusters)
    is_dublicate = sum(groups) - 1
    return is_dublicate

In [43]:
embeddings = (df.name_1_minhash[0], df.name_2_minhash[0])
print(embeddings)
clusters = lsh_clustering(embeddings, seed=1)
groups = get_group_indices(clusters)
print(groups)

(array([2837697543, 4286705991, 4033516562, 1576884619, 1735179891,
        310656841, 3408608597,   13678415,  301410491, 2953523810,
        477352457,  141669305, 3029537362, 2001883836, 2949141549,
       1565102088,  397218730, 1395921778, 1829472766, 2079612969,
        563933777, 3915858165, 1827382166, 4244790857, 3556267963,
        463641213, 4068020223, 1681809753, 3530864373, 2202535791,
       2805053486, 1252092394, 1632445077, 1385409291, 1593987726,
       3225096065, 2003495294, 2399629907, 1062683937, 1994360923,
       1037929784, 2059782829, 2747494584,  132995104, 2948986761,
       2914122668, 1406448581, 3796494225, 3255852362, 2154365917,
       3218537268,  516970915, 2343596022, 3070107276,   77538504,
        288992157, 1330442140,  877753739, 4109425954,  841356568,
       1023452212, 1485945366,  458509675, 4091971844, 2023237090,
        764718198,  115107191,  754736214, 2373010499, 3601345872,
       3242009781, 1266357576,  517701005,  387087565, 280265

In [51]:
df.head()

,pair_id,name_1,name_2,is_duplicate,fuzz_ratio,fuzz_part_ratio,fuzz_set_ratio,fuzz_sort_ratio,name_1_simhash,name_2_simhash,name_1_minhash,name_2_minhash
0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0,58,68,59,55,8280139831965439638,1189005557588197376,"[2837697543, 4286705991, 4033516562, 157688461...","[2269330533, 1950295083, 2747852593, 821000859..."
1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0,62,65,78,59,3467845109336244224,10454710756391821586,"[383969271, 2548742056, 266325389, 3073401718,...","[41399270, 2875731467, 1607583424, 1048751950,..."
2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0,6,67,12,12,2342489737136587522,17894649692736411033,"[3402606397, 499673567, 298620726, 3214740442,...","[509944821, 3511746368, 997396335, 3279385, 78..."
3,4,Powermax Rubber Factory,Co. One,0,13,29,34,34,7143311180216527003,15600328471728316353,"[1119473718, 2434485798, 3411428267, 333324073...","[2191618327, 1006052587, 3887610657, 169907728..."
4,5,Tress A/S,Longyou Industries Park Zhejiang,0,29,67,24,24,2631219134025501574,12160026900651802638,"[3379829488, 1580889199, 2105789094, 363853213...","[1377538848, 102079391, 1531451454, 1667699962..."


In [ ]:
df['lsh_clusters'] = df.apply(lambda x: get_groups(x.name_1_minhash, x.name_2_minhash), axis=1)

In [5]:
from text_dedup.near_dedup import MinHashEmbedder
from text_dedup.postprocess import lsh_clustering
from text_dedup.postprocess import get_group_indices

corpus = [
    "The quick brown fox jumps over the lazy dog",
    "The quick brown fox jumps over the lazy dog",
    "This is a test",
    "This is a test",
]

embedder = MinHashEmbedder()
embeddings = embedder.embed(corpus)

clusters = lsh_clustering(embeddings, seed=1)
groups = get_group_indices(clusters)
print(groups)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [6]:
distinct = set(df.name_1.values)
distinct2 = set(df.name_2.values)
all_distinct = distinct.union(distinct2)

In [43]:
embedder = MinHashEmbedder(16)
embeddings = embedder.embed(all_distinct)

clusters = lsh_clustering(embeddings, seed=1, num_perm=16)
groups = get_group_indices(clusters)
print(groups)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [44]:
cluster_dict = {item:group for item, group in zip(all_distinct, groups)}

In [45]:
embedding_dict = {item:embedding for item, embedding in zip(all_distinct, embeddings)}

In [46]:
cluster_dict

{'Premier Logistics Solutions': 0,
 'Dynasty Freight Consolidators': 1,
 'Iguasport Ltd.': 2,
 'Llc Octoblue Russian Federation': 3,
 'Pirelli Neumaticos Argentina Sa': 3120,
 'Continental Tire D Mexico': 5,
 'Surya Handicrafts': 6,
 'Meen Been Elastomers Pvt., Ltd.': 7,
 'Wuhan Mecaplast Co., Ltd.': 1992,
 'Dalian Rongxin Garment Co., Ltd.': 9,
 'Dip Plc': 10,
 'Suzhou Toptex Mechatronics Co., Ltd.': 11,
 'Itg Automotive Safety Components International S.A De C.V.': 12,
 'Asr Resin & Chemicals Pvt., Ltd.': 199,
 'Imran Machinery Store': 14,
 'Dongguan City Thousands Of Horses Imp. & Exp. . Co.': 15,
 'Ge Power India': 16,
 'Rana Motors And Metal Works Eng Co.': 17,
 'Bitumar Sorel Tracy Qc': 18,
 'Journeys Division Of Gco Canada Inc.': 19,
 'Productos Y Servicios Del Centro S.': 20,
 'Mogul Teks San Ve Tic Ltd. Sti.': 21,
 'Surishopping': 22,
 'To The Order Of First Commercial': 23,
 'MANEKI JAPAN CO., LIMITED': 24,
 'Hsbc Bank Oman Saog': 25,
 'Atc Tires Pvt., Ltd.': 1992,
 'Aipl Mar

In [47]:
cluster_df = pd.DataFrame.from_dict(cluster_dict,orient='index',columns=['group'])

In [48]:
cluster_df.to_csv('data/clusters_16_df.csv')

In [49]:
emb_df = pd.DataFrame.from_dict(embedding_dict, orient='index')

In [50]:
emb_df.to_csv('data/emb_16_df.csv')

In [51]:
len(set(groups))

14521

In [52]:
len(all_distinct)

18022

In [15]:
from deep_translator import (GoogleTranslator,
                             PonsTranslator,
                             LingueeTranslator,
                             MyMemoryTranslator,
                             YandexTranslator,
                            #  DeepL,
                            #  QCRI,
                             single_detection,
                             batch_detection)

In [53]:
translated_raw = GoogleTranslator('auto', 'en').translate_batch(all_distinct)

In [58]:
translated_dict = {item:transl for item, transl in zip(all_distinct, translated_raw)}

In [59]:
translated_df = pd.DataFrame.from_dict(translated_dict,orient='index', columns=['transl'])

In [63]:
translated_df.head(30)

,transl
Premier Logistics Solutions,Premier Logistics Solutions
Dynasty Freight Consolidators,Dynasty Freight Consolidators
Iguasport Ltd.,Iguasport Ltd.
Llc Octoblue Russian Federation,Llc Octoblue Russian Federation
Pirelli Neumaticos Argentina Sa,Pirelli Tires Argentina Sa
Continental Tire D Mexico,Continental Tire D Mexico
Surya Handicrafts,Surya Handicrafts
"Meen Been Elastomers Pvt., Ltd.","Meen Leg Elastomers Pvt., Ltd."
"Wuhan Mecaplast Co., Ltd.","Wuhan Mecaplast Co., Ltd."
"Dalian Rongxin Garment Co., Ltd.","Dalian Rongxin Garment Co., Ltd."


In [64]:
translated_df.to_csv('data/translated_df.csv')

In [65]:
embedder = MinHashEmbedder()
embeddings = embedder.embed(translated_raw)

clusters = lsh_clustering(embeddings, seed=1) #, num_perm=16
groups = get_group_indices(clusters)
print(groups)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [66]:
print(len(set(groups)))

14894

In [75]:
cluster_dict = {item:group for item, group in zip(translated_raw, groups)}
embedding_dict = {item:embedding for item, embedding in zip(translated_raw, embeddings)}
cluster_df = pd.DataFrame.from_dict(cluster_dict,orient='index',columns=['group'])
emb_df = pd.DataFrame.from_dict(embedding_dict, orient='index')

In [76]:
cluster_df.to_csv('data/clusters_translated_df.csv')
emb_df.to_csv('data/emb_translated_df.csv')

In [77]:
embedder = MinHashEmbedder(32)
embeddings = embedder.embed(translated_raw)

clusters = lsh_clustering(embeddings, seed=1, num_perm=32) #, num_perm=16
groups = get_group_indices(clusters)
print(len(set(groups)))
cluster_dict = {item:group for item, group in zip(translated_raw, groups)}
embedding_dict = {item:embedding for item, embedding in zip(translated_raw, embeddings)}
cluster_df = pd.DataFrame.from_dict(cluster_dict,orient='index',columns=['group'])
emb_df = pd.DataFrame.from_dict(embedding_dict, orient='index')
cluster_df.to_csv('data/clusters_translated_32_df.csv')
emb_df.to_csv('data/emb_translated_32_df.csv')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [78]:
embedder = MinHashEmbedder(16)
embeddings = embedder.embed(translated_raw)

clusters = lsh_clustering(embeddings, seed=1, num_perm=16) #, num_perm=16
groups = get_group_indices(clusters)
print(len(set(groups)))
cluster_dict = {item:group for item, group in zip(translated_raw, groups)}
embedding_dict = {item:embedding for item, embedding in zip(translated_raw, embeddings)}
cluster_df = pd.DataFrame.from_dict(cluster_dict,orient='index',columns=['group'])
emb_df = pd.DataFrame.from_dict(embedding_dict, orient='index')
cluster_df.to_csv('data/clusters_translated_16_df.csv')
emb_df.to_csv('data/emb_translated_16_df.csv')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [79]:
embedder = MinHashEmbedder(8)
embeddings = embedder.embed(translated_raw)

clusters = lsh_clustering(embeddings, seed=1, num_perm=8) #, num_perm=16
groups = get_group_indices(clusters)
print(len(set(groups)))
cluster_dict = {item:group for item, group in zip(translated_raw, groups)}
embedding_dict = {item:embedding for item, embedding in zip(translated_raw, embeddings)}
cluster_df = pd.DataFrame.from_dict(cluster_dict,orient='index',columns=['group'])
emb_df = pd.DataFrame.from_dict(embedding_dict, orient='index')
cluster_df.to_csv('data/clusters_translated_8_df.csv')
emb_df.to_csv('data/emb_translated_8_df.csv')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b045afd9-a76f-4856-9836-14ff20b55163' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>